In [1]:
import math
import numpy as np
import pandas as pd
from random import random
from numpy import array
from numpy import cumsum
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed,AveragePooling1D,Flatten

Using TensorFlow backend.


In [16]:
!pip install NiaPy

     |████████████████████████████████| 61kB 1.7MB/s 
     |████████████████████████████████| 143kB 5.7MB/s 
     |████████████████████████████████| 50.0MB 869kB/s 
     |████████████████████████████████| 13.8MB 19.5MB/s 
     |████████████████████████████████| 71kB 18.9MB/s 
ERROR: spacy 2.0.18 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
ERROR: imgaug 0.2.9 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
ERROR: fastai 1.0.52 has requirement numpy>=1.15, but you'll have numpy 1.14.6 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: cvxpy 1.0.15 has requirement scipy>=1.1.0, but you'll have scipy 1.0.1 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.16.3
    Uninstalling numpy-1.16.3:
      Succe

In [2]:
!wget https://www.dropbox.com/s/2qkkea4nxbzh7w1/outcomes.csv
!wget https://www.dropbox.com/s/rppw0q78nk03644/patient_data_24_hours.csv

--2019-05-23 08:12:08--  https://www.dropbox.com/s/rppw0q78nk03644/patient_data_24_hours.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:6031:1::a27d:5101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/rppw0q78nk03644/patient_data_24_hours.csv [following]
--2019-05-23 08:12:08--  https://www.dropbox.com/s/raw/rppw0q78nk03644/patient_data_24_hours.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc1c7cf587874a6afb5d4f1c9a3f.dl.dropboxusercontent.com/cd/0/inline/Aha-tNs3zHvUZnBYtAP4sJFfLh0f4n0_a9d2h2tN-Xc61-VXi5uJPJ2aQ5iot-hSzsn5WHutPoWa8pmFJeVmOk5tIR0TrbI_qxUXV6ghiYPvvA/file# [following]
--2019-05-23 08:12:09--  https://uc1c7cf587874a6afb5d4f1c9a3f.dl.dropboxusercontent.com/cd/0/inline/Aha-tNs3zHvUZnBYtAP4sJFfLh0f4n0_a9d2h2tN-Xc61-VXi5uJPJ2aQ5iot-hSzsn5WHutPoWa8pmFJeVmOk5tIR0TrbI

In [4]:
df = pd.read_csv('../finalset/patient_data_24_hours.csv')
df.head(50)

,SUBJECT_ID,TimeStamp,Albumin,Alk. Phosphate,ALT,AST,Total Bili,BUN,Cholesterol,Creatinine,...,Platelets,Respiratory Rate,SaO2,Arterial BP [Systolic],Temperature C,TroponinI,TroponinT,Urine,WBC,Previous WeightF
0,17,0 days 00:00:00.000000000,0.0,0.000000,0.000000,0.000000,0.0,0.060606,0.0,0.013433,...,0.103116,0.143564,0.990099,0.445489,0.0,0.0,0.0,0.0,0.021739,0.000000
1,17,1 days 00:00:00.000000000,0.0,0.000000,0.000000,0.000000,0.0,0.055556,0.0,0.012687,...,0.177300,0.247525,0.990099,0.387218,0.0,0.0,0.0,0.0,0.047259,0.000000
2,17,0 days 23:00:00.000000000,0.0,0.000000,0.000000,0.000000,0.0,0.055556,0.0,0.012687,...,0.177300,0.247525,0.990099,0.387218,0.0,0.0,0.0,0.0,0.047259,0.000000
3,17,0 days 22:00:00.000000000,0.0,0.000000,0.000000,0.000000,0.0,0.055556,0.0,0.012687,...,0.177300,0.287129,0.990099,0.372180,0.0,0.0,0.0,0.0,0.047259,0.000000
4,17,0 days 21:00:00.000000000,0.0,0.000000,0.000000,0.000000,0.0,0.055556,0.0,0.012687,...,0.177300,0.267327,0.990099,0.357143,0.0,0.0,0.0,0.0,0.047259,0.000000
5,17,0 days 20:00:00.000000000,0.0,0.000000,0.000000,0.000000,0.0,0.055556,0.0,0.012687,...,0.177300,0.316832,0.990099,0.334586,0.0,0.0,0.0,0.0,0.047259,0.000000
6,17,0 days 19:00:00.000000000,0.0,0.000000,0.000000,0.000000,0.0,0.055556,0.0,0.012687,...,0.177300,0.237624,0.990099,0.383459,0.0,0.0,0.0,0.0,0.047259,0.000000
7,17,0 days 17:00:00.000000000,0.0,0.000000,0.000000,0.000000,0.0,0.055556,0.0,0.012687,...,0.177300,0.262376,0.990099,0.300752,0.0,0.0,0.0,0.0,0.047259,0.000000
8,17,0 days 16:00:00.000000000,0.0,0.000000,0.000000,0.000000,0.0,0.055556,0.0,0.012687,...,0.177300,0.198020,0.990099,0.327068,0.0,0.0,0.0,0.0,0.047259,0.000000
9,17,0 days 15:00:00.000000000,0.0,0.000000,0.000000,0.000000,0.0,0.055556,0.0,0.012687,...,0.177300,0.217822,0.990099,0.421053,0.0,0.0,0.0,0.0,0.047259,0.000000


In [5]:
matrix3D = np.array(df.drop(['SUBJECT_ID', 'TimeStamp'], 1))
matrix3D

array([[0.        , 0.        , 0.        , ..., 0.        , 0.02173913,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.04725898,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.04725898,
        0.        ],
       ...,
       [0.79032258, 0.        , 0.00266276, ..., 0.        , 0.01587902,
        0.        ],
       [0.79032258, 0.        , 0.00266276, ..., 0.        , 0.01039698,
        0.        ],
       [0.79032258, 0.        , 0.00266276, ..., 0.        , 0.01587902,
        0.        ]])

In [6]:
matrix3D.shape

(164675, 37)

In [7]:
matrix3D = np.array(matrix3D).reshape((6587, 25, 37))
matrix3D.shape

(6587, 25, 37)

In [6]:

outcomes = pd.read_csv('outcomes.csv')
outcomes.head(10)

,SUBJECT_ID,LABEL
0,17,0
1,21,0
2,23,0
3,34,0
4,36,1
5,61,0
6,68,1
7,85,0
8,94,0
9,103,1


In [7]:
Y = np.array(outcomes.drop(['SUBJECT_ID'], 1))
Y.shape

(6587, 1)

In [8]:
X = matrix3D
X.shape

(6587, 25, 37)

In [9]:
X_train = X[:5270]
X_test = X[5270:]
Y_train = Y[:5270]
Y_test = Y[5270:]
display(X_train.shape)
display(X_test.shape)
display(Y_train.shape)
display(Y_test.shape)

(5270, 25, 37)

(1317, 25, 37)

(5270, 1)

(1317, 1)

In [12]:
model = Sequential()
model.add(LSTM(37, input_shape=(25, 37),return_sequences=True))
model.add(TimeDistributed(Dense(1)))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['acc'])
print(model.summary())
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=30,batch_size=8,verbose=1)

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 25, 37)            11100     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 25, 1)             38        
_________________________________________________________________
flatten_1 (Flatten)          (None, 25)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 26        
Total params: 11,164
Trainable params: 11,164
Non-trainable params: 0
_________________________________________________________________
None
Instructions for updating:
Use tf.cast instead.
Train on 5270 samples, validate on 1317 samples
Epoch 1/30
1424/5270 [=======>......................] - ETA: 36s - loss: 0.2346 - acc: 0.6208

KeyboardInterrupt: ignored

In [0]:
scores = model.evaluate(X_test, Y_test)
scores[1]*100

1317/1317 [==============================] - 0s 138us/step


55.884586189765315

In [0]:
X_trainz = (X_train - np.mean(X_train))/np.std(X_train)
X_trainz

array([[[-0.52433825, -0.52433825, -0.52433825, ..., -0.52433825,
         -0.43351284, -0.52433825],
        [-0.52433825, -0.52433825, -0.52433825, ..., -0.52433825,
         -0.32689171, -0.52433825],
        [-0.52433825, -0.52433825, -0.52433825, ..., -0.52433825,
         -0.32689171, -0.52433825],
        ...,
        [-0.52433825, -0.52433825, -0.52433825, ..., -0.52433825,
         -0.32689171, -0.52433825],
        [-0.52433825, -0.52433825, -0.52433825, ..., -0.52433825,
         -0.32689171, -0.52433825],
        [-0.52433825, -0.52433825, -0.52433825, ..., -0.52433825,
         -0.32689171, -0.52433825]],

       [[-0.52433825, -0.45007681, -0.45536366, ..., -0.52433825,
         -0.34979551,  0.43185661],
        [-0.52433825, -0.45007681, -0.45536366, ..., -0.52433825,
         -0.34979551,  0.43185661],
        [-0.52433825, -0.45007681, -0.45536366, ..., -0.52433825,
         -0.34979551,  0.43185661],
        ...,
        [-0.52433825, -0.45007681, -0.45536366, ..., -

In [0]:
X_testz = (X_test - np.mean(X_test))/np.std(X_test)
X_testz

array([[[ 6.16927856, -0.29609609, -0.28211854, ..., -0.29609609,
         -0.29609609, -0.29609609],
        [ 6.16927856, -0.29609609, -0.28211854, ..., -0.29609609,
         -0.29609609, -0.29609609],
        [ 6.16927856, -0.29609609, -0.28211854, ..., -0.29609609,
         -0.29609609, -0.29609609],
        ...,
        [ 6.16927856, -0.29609609, -0.28211854, ..., -0.29609609,
         -0.29609609, -0.29609609],
        [ 6.16927856, -0.29609609, -0.28211854, ..., -0.29609609,
         -0.29609609, -0.29609609],
        [ 6.16927856, -0.29609609, -0.28211854, ..., -0.29609609,
         -0.29609609, -0.29609609]],

       [[ 6.63109103, -0.29609609, -0.2770358 , ..., -0.29609609,
         -0.03809802, -0.29609609],
        [ 6.47715354, -0.29609609, -0.27322374, ..., -0.29609609,
         -0.03809802, -0.29609609],
        [ 6.47715354, -0.29609609, -0.27322374, ..., -0.29609609,
         -0.03809802, -0.29609609],
        ...,
        [ 6.47715354, -0.29609609, -0.27322374, ..., -

In [0]:
from keras.layers import GRU
modelGRU = Sequential()
modelGRU.add(GRU(100, activation='tanh', recurrent_activation='hard_sigmoid', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0, implementation=1, reset_after=False,input_shape=(25, 37)))
modelGRU.add(Dense(1, activation='sigmoid'))
modelGRU.compile(loss='mean_squared_error', optimizer='adam', metrics=['acc'])
print(modelGRU.summary())
modelGRU.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=30,batch_size=8,verbose=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_3 (GRU)                  (None, 100)               41400     
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 101       
Total params: 41,501
Trainable params: 41,501
Non-trainable params: 0
_________________________________________________________________
None
Train on 5270 samples, validate on 1317 samples
Epoch 1/30
5270/5270 [==============================] - 11s 2ms/step - loss: 0.2269 - acc: 0.6528 - val_loss: 0.2465 - val_acc: 0.5740
Epoch 2/30
5270/5270 [==============================] - 10s 2ms/step - loss: 0.2220 - acc: 0.6577 - val_loss: 0.2456 - val_acc: 0.5740
Epoch 3/30
5270/5270 [==============================] - 10s 2ms/step - loss: 0.2220 - acc: 0.6562 - val_loss: 0.2450 - val_acc: 0.5740
Epoch 4/30
5270/5270 [==============================] - 9s 2ms/step - loss: 

In [0]:
scores = modelGRU.evaluate(X_testz, Y_test)
scores[1]*100

1317/1317 [==============================] - 1s 560us/step


49.12680334318924

In [0]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [0]:
ans = []
class LSTM(object):
    def __init__(self):
        # define lower bound of benchmark function
        self.Lower = 0
        # define upper bound of benchmark function
        self.Upper = 1

    # function which returns evaluate function
    def function(self):
        def evalute(D,sol):
            sol = np.array(sol)
            sol = sigmoid(sol)
            print(sol)
            #display(sol)
            op = sol>=0.50
            X_tr = X_train[:,:,op]
            X_te = X_test[:,:,op]
            print(op)
            from keras.models import Sequential
            from keras.layers import GRU,Dense,LSTM,TimeDistributed
            model = Sequential()
            model.add(LSTM(37, input_shape=(25, 37),return_sequences=True))
            model.add(TimeDistributed(Dense(1)))
            model.add(Flatten())
            model.add(Dense(1, activation='sigmoid'))
            model.compile(loss='mean_squared_error', optimizer='adam', metrics=['acc'])
            model.fit(X_tr, Y_train, validation_data=(X_te, Y_test), epochs=3,batch_size=50,verbose=0)
            scores = model.evaluate(X_te, Y_test, verbose=0)
            print(scores)
            k = -1*scores[1]
            return k
        return evalute

In [24]:
from NiaPy.algorithms.basic import BatAlgorithm
for i in range(5):
    algorithm = BatAlgorithm(D=37, NP=370, nFES=5,A = 0.5,r=0.5,Qmin=0.0,Qmax =2.0, benchmark=LSTM())
    best = algorithm.run()
    print(best[1]*-100)
    ans.append(-1*best[1])

[0.60435648 0.55652579 0.51290082 0.55313473 0.62771598 0.62483347
 0.57277219 0.5246291  0.66456548 0.68490907 0.62404571 0.63458407
 0.71355039 0.65978897 0.51923699 0.70869549 0.72813613 0.55711038
 0.72236398 0.51465741 0.5515607  0.57471537 0.55662697 0.58766809
 0.55592764 0.50812924 0.57565023 0.58150564 0.57464121 0.71750048
 0.5901073  0.62714198 0.52404396 0.55072019 0.65157123 0.72013204
 0.71331529]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True]
[0.24558566118065117, 0.574031890751108]
[0.5216684  0.56046179 0.67827447 0.68941771 0.5780872  0.71256205
 0.57794584 0.60878305 0.64560053 0.57309581 0.62403119 0.72276823
 0.72456377 0.68883632 0.67177508 0.66071236 0.66581959 0.56865665
 0.52348191 0.59864459 0.68054883 0.62070669 0.66999382 0.55349603
 0.57621251 0.72622379 0.68728721 0.54374439 0.62

KeyboardInterrupt: ignored

In [0]:
ans

[0.574031890751108,
 0.5717539864230898,
 0.5717539864230898,
 0.5702353835377444,
 0.5732725893084353]

In [0]:
import logging
from scipy.spatial.distance import euclidean
from numpy import full, apply_along_axis, argmin, copy, sum, inf, fmax, pi, where

logging.basicConfig()
logger = logging.getLogger('NiaPy.algorithms.basic')
logger.setLevel('INFO')

__all__ = ['GlowwormSwarmOptimization', 'GlowwormSwarmOptimizationV1', 'GlowwormSwarmOptimizationV2', 'GlowwormSwarmOptimizationV3']

class GlowwormSwarmOptimization():
	r"""Implementation of glowwarm swarm optimization.
	**Algorithm:** Glowwarm Swarm Optimization Algorithm
	**Date:** 2018
	**Authors:** Klemen Berkovič
	**License:** MIT
	**Reference URL:** https://www.springer.com/gp/book/9783319515946
	**Reference paper:** Kaipa, Krishnanand N., and Debasish Ghose. Glowworm swarm optimization: theory, algorithms, and applications. Vol. 698. Springer, 2017.
	"""
	def __init__(self,D, l0,nt,rho,gamma,beta,s,lower,upper):
		self.D, self.n, self.l0, self.nt, self.rho, self.gamma, self.beta, self.s,self.Lower,self.Upper = D,n, l0, nt, rho, gamma, beta, s,lower,upper
    self.bRange = self.Upper - self.Lower

	def randMove(self, i):
		j = i
		while i == j: j = self.randint(self.n)
		return j

	def getNeighbors(self, i, r, GS, L):
		N = full(self.n, 0)
		for j, gw in enumerate(GS): N[j] = 1 if i != j and euclidean(GS[i], gw) <= r and L[i] >= L[j] else 0
		return N

	def probabilityes(self, i, N, L):
		d, P = sum(L[where(N == 1)] - L[i]), full(self.n, .0)
		for j in range(self.n): P[i] = ((L[j] - L[i]) / d) if N[j] == 1 else 0
		return P

	def moveSelect(self, pb, i):
		r, b_l, b_u = self.rand(), 0, 0
		for j in range(self.n):
			b_l, b_u = b_u, b_u + pb[i]
			if b_l < r < b_u: return j
		return self.randint(self.n)

	def calcLuciferin(self, L, GS_f): return (1 - self.rho) * L + self.gamma * GS_f

	def rangeUpdate(self, R, N, rs): return R + self.beta * (self.nt - sum(N))

	def getBest(self, GS, GS_f, xb, xb_f):
		ib = argmin(GS_f)
		if GS_f[ib] < xb_f: return GS[ib], GS_f[ib]
		else: return xb, xb_f

	def runTask(self):
		rs = euclidean(full(self.D, 0), self.bRange)
		GS, GS_f, L, R = self.uniform(self.Lower, self.Upper, [self.n, self.D]), full(self.n, inf), full(self.n, self.l0), full(self.n, rs)
		xb, xb_f = None, inf
		for i in range(5):
			GSo, Ro, GS_f = copy(GS), copy(R), apply_along_axis(task.eval, 1, GS)
			xb, xb_f = self.getBest(GS, GS_f, xb, xb_f)
			L = self.calcLuciferin(L, GS_f)
			N = [self.getNeighbors(i, Ro[i], GSo, L) for i in range(self.n)]
			P = [self.probabilityes(i, N[i], L) for i in range(self.n)]
			j = [self.moveSelect(P[i], i) for i in range(self.n)]
			for i in range(self.n): GS[i] = task.repair(GSo[i] + self.s * ((GSo[j[i]] - GSo[i]) / (euclidean(GSo[j[i]], GSo[i]) + 1e-31)))
			for i in range(self.n): R[i] = max(0, min(rs, self.rangeUpdate(Ro[i], N[i], rs)))
		return xb, xb_f


In [0]:
import numpy as np
def sigmoid(x):
    return 1/(1+np.exp(-x))

import random

def Fun(D, sol):
    sol = np.array(sol)
    #print(sol)
    sol = sigmoid(sol)
    op = sol>=0.6
    #print(op)
    X_tr = X_train_tf[:,op]
    X_valid = X_valid_tf[:,op]
    print(X_valid.shape)
    model=LogisticRegression(C = 0.01, penalty = 'l2', solver = 'lbfgs' , random_state = 42)
    model.fit(X_tr, y_train)
    y_train_preds = model.predict_proba(X_tr)[:,1]
    y_valid_preds = model.predict_proba(X_valid)[:,1]
    auc_train = roc_auc_score(y_train, y_train_preds)
    auc_valid = roc_auc_score(y_valid, y_valid_preds)
    print(f'auc_valid ={auc_valid}')
    k = -1*(auc_valid)
    return k